# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.feature as gf

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")
# Does the same thing as index_col=
# city_data_df.set_index("City_ID", inplace=True)

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,katsuren-haebaru,26.3369,127.8719,79.09,95,20,6.91,JP,1731184160
1,port mathurin,-19.6833,63.4167,75.72,88,35,16.17,MU,1731184162
2,adamstown,-25.0660,-130.1015,73.29,87,99,19.86,PN,1731184164
3,geretsried,47.8578,11.4805,40.28,94,81,0.92,DE,1731184166
4,blackmans bay,-43.0167,147.3167,51.93,73,88,1.99,AU,1731184168


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
#Assistance on parameters using Xpert Learning Assistance
points = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    size='Humidity',  
    color='City',
    alpha=0.7,  
    title='City Humidity Map',
    geo=True,
    tiles='OSM')
#features = gv.borders #gv.FeatureCollection
map_plot = points
map_plot.opts(
    width=1000,
    height=600,
    tools=['hover'])
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df["Max Temp"] > 75) & (city_data_df["Max Temp"] < 90) &  
                              (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] < 20) &    
                              (city_data_df["Humidity"] < 50)]

# Drop any rows with null values
ideal_cities_df.dropna(how="any")

# Display sample data
ideal_cities_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
227,cabo san lucas,22.8909,-109.9124,81.75,49,0,9.22,MX,1731184572
232,mandiana,10.6333,-8.6833,78.55,43,4,2.35,GN,1731184632
237,san patricio,28.0170,-97.5169,85.95,44,0,8.59,US,1731184646
253,tegueste,28.5167,-16.3167,76.35,32,0,4.00,ES,1731184677
304,alpine,32.8350,-116.7664,77.14,14,0,4.61,US,1731184784
398,ojinaga,29.5667,-104.4167,78.57,11,0,0.00,MX,1731184974
426,rio verde de mato grosso,-18.9181,-54.8442,87.48,46,12,4.70,BR,1731185030
532,ocampo,21.6500,-101.5000,76.95,27,12,5.82,MX,1731185257
550,crucecita,-28.9500,-58.9833,82.62,38,0,9.40,AR,1731185297


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
227,cabo san lucas,MX,22.8909,-109.9124,49,
232,mandiana,GN,10.6333,-8.6833,43,
237,san patricio,US,28.0170,-97.5169,44,
253,tegueste,ES,28.5167,-16.3167,32,
304,alpine,US,32.8350,-116.7664,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
limit = 1


params = {
    "categories":"accommodation.hotel",
    "limit":limit,
    "apiKey":geoapify_key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
mandiana - nearest hotel: Hôtel Prince
san patricio - nearest hotel: No hotel found
tegueste - nearest hotel: Hotel Aguere
alpine - nearest hotel: Ayres Lodge Alpine
ojinaga - nearest hotel: Hotel
rio verde de mato grosso - nearest hotel: Balneário e Pousada Paraíso
ocampo - nearest hotel: No hotel found
crucecita - nearest hotel: No hotel found
gabu - nearest hotel: HOTEL VISIOM


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
227,cabo san lucas,MX,22.8909,-109.9124,49,Comfort Rooms
232,mandiana,GN,10.6333,-8.6833,43,Hôtel Prince
237,san patricio,US,28.0170,-97.5169,44,No hotel found
253,tegueste,ES,28.5167,-16.3167,32,Hotel Aguere
304,alpine,US,32.8350,-116.7664,14,Ayres Lodge Alpine
398,ojinaga,MX,29.5667,-104.4167,11,Hotel
426,rio verde de mato grosso,BR,-18.9181,-54.8442,46,Balneário e Pousada Paraíso
532,ocampo,MX,21.6500,-101.5000,27,No hotel found
550,crucecita,AR,-28.9500,-58.9833,38,No hotel found


In [ ]:
#Filter hotel_df to only show cities that returned back a hotel
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
227,cabo san lucas,MX,22.8909,-109.9124,49,Comfort Rooms
232,mandiana,GN,10.6333,-8.6833,43,Hôtel Prince
253,tegueste,ES,28.5167,-16.3167,32,Hotel Aguere
304,alpine,US,32.8350,-116.7664,14,Ayres Lodge Alpine
398,ojinaga,MX,29.5667,-104.4167,11,Hotel
426,rio verde de mato grosso,BR,-18.9181,-54.8442,46,Balneário e Pousada Paraíso
555,gabu,GW,12.2833,-14.2167,38,HOTEL VISIOM


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [76]:
%%capture --no-display

# Configure the map plot
points = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    size='Humidity',  
    color='City',
    alpha=0.7,  
    title='My Ideal City Map with a Hotel',
    geo=True,
    tiles='OSM',
    xlim=(-170, 170),
    ylim=(-55, 75),
    hover_cols= ["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"])

map_plot = points
map_plot.opts(
    width=1000,
    height=600)
    #tools=['hover'])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)